In [1]:
from faster_whisper import WhisperModel
from transformers import pipeline
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
import gradio as gr
import sentencepiece
import openai
from dotenv import load_dotenv
import os

load_dotenv()

/home/surfer/anaconda3/envs/TTT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [20]:
# Retrieve the OpenAI API key from the environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Ensure that the API key is loaded
if not openai.api_key:
    raise ValueError("OpenAI API key is not set in the environment variables.")

## Faster Whisper for Audio ##


In [4]:
### 1. Faster Whisper for Audio Transcription ###
def transcribe_audio(audio_file):
    model = WhisperModel("small", device="cuda")  # Use "cuda" if you have a GPU
    segments, _ = model.transcribe(audio_file, language="es")
    transcription = " ".join([segment.text for segment in segments])
    return transcription

## 2. Translation Pipelines ###

In [5]:

# 2. Translation Pipelines
translator_es_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en", device=0)
translator_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es", device=0)


Device set to use cuda:0
Device set to use cuda:0


In [6]:
# translate from spanish to english
def translate_to_english(text):
    translated = translator_es_en(text, max_length=512)
    return translated[0]['translation_text']

# Translate from english to spanish
def translate_to_spanish(text):
    translated = translator_en_es(text, max_length=512)
    return translated[0]['translation_text']


## 3. RAG Setup (Using LangChain and FAISS)


In [ ]:
# Create the vector store and embedding.
def create_vector_db(resume_file):
    loader = TextLoader(resume_file)
    documents = loader.load()
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

# Generate the response wtih gpt-4o-mini
def generate_response(question, vectorstore):
    llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0.7)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())
    return qa_chain.inoke(question)

# Load the RAG Knowledge Base
vectorstore = create_vector_db("Resume.txt")


## 4. Complete Pipeline

In [ ]:
def process_pipeline(audio_file):
    # Step 1: Transcribe audio (Spanish)
    spanish_text = transcribe_audio(audio_file)
    
    # Step 2: Translate Spanish transcription to English
    english_query = translate_to_english(spanish_text)
    
    # Step 3: Query the RAG system in English
    english_response = generate_response(english_query, vectorstore)
    
    # Step 4: Translate the English response back to Spanish
    spanish_response = translate_to_spanish(english_response)
    
    return spanish_text[0], spanish_response[0]


# 5. Gradio Frontend

In [47]:
interface = gr.interface = gr.Interface(
    fn=process_pipeline,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs=["text", "text"],
    title="Spanish Q&A System",
    description="Ask questions in Spanish via audio, and the system will respond based on your resume.",
    )


# Launch the app
if __name__ == "__main__":
    interface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/surfer/anaconda3/envs/TTT/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/surfer/anaconda3/envs/TTT/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/surfer/anaconda3/envs/TTT/lib/python3.11/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/surfer/anaconda3/envs/TTT/lib/python3.11/site-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/surfer/anaconda3/envs/TTT/lib/python3.11/site-packages/anyio/to_thread.p